In [5]:
import csv
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

base_url = 'https://songdata.io/'
search_url = urljoin(base_url, 'search?query=label%3Akorea')

# 데이터를 저장할 CSV 파일을 엽니다.
with open('output.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # 헤더 행을 작성합니다.
    writer.writerow(['Name', 'Artist', 'Key', 'BPM'])
    
    url = search_url
    while url:
        response = requests.get(url)

        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')

            # 테이블의 각 행에 대한 정보를 가져옵니다.
            rows = soup.select('tr.table_object')
            
            for row in rows:
                # 각 행에서 필요한 정보를 추출합니다.
                name = row.select_one('td.table_name').text.strip()
                artist = row.select_one('td.table_artist').text.strip()
                key = row.select_one('td.table_key').text.strip()
                bpm = row.select_one('td.table_bpm').text.strip()

                # 추출한 정보를 CSV 파일에 작성합니다.
                writer.writerow([name, artist, key, bpm])
            
            # Next Page 버튼의 링크를 찾습니다.
            next_page = soup.select_one('span.search_page a')
            if next_page:
                url = urljoin(base_url, next_page['href'])
                # next 버튼을 누르기 전에 5초 동안 대기합니다.
                time.sleep(5)
            else:
                url = None
